# setup

In [1]:
base_url="http://host.docker.internal:11434"
model_name = "gemma3:27b"
from langchain_ollama import ChatOllama
model = ChatOllama(
    model=model_name,
    base_url=base_url,
)

# tools

In [3]:
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig

class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""
    name: str
    arguments: Dict[str, Any]
    id: str
    type: str
    


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    
    return requested_tool.invoke(tool_call_request["args"], config=config)

In [8]:
from langchain_core.tools import tool


@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together."""
    return a * b


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


@tool
def retrieve(input_query: str) -> str:
    """
    Retrieve useful and important documents based on the input query.

    This function is intended to interact with a retrieval system, such as a vector store,
    keyword-based search, or document index, to return relevant content.
    """
    return input_query

tools = [retrieve]

print(retrieve.args_schema.model_json_schema())

{'description': 'Retrieve useful and important documents based on the input query.\n\nThis function is intended to interact with a retrieval system, such as a vector store,\nkeyword-based search, or document index, to return relevant content.', 'properties': {'input_query': {'title': 'Input Query', 'type': 'string'}}, 'required': ['input_query'], 'title': 'retrieve', 'type': 'object'}


In [37]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

retrieve(input_query: str) -> str - Retrieve useful and important documents based on the input query.

This function is intended to interact with a retrieval system, such as a vector store,
keyword-based search, or document index, to return relevant content.


# single tool calling at a time (deprecated)

In [38]:
system_prompt = f"""\
You have access to functions. If you decide to invoke any of the function(s),
you MUST put it in the format of
{{{{
    "name": function name, 
    "args": dictionary of argument name and its value,
}}}}

You SHOULD NOT include any other text in the response if you call a function
{rendered_tools}

"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

print(system_prompt)

You have access to functions. If you decide to invoke any of the function(s),
you MUST put it in the format of
{{
    "name": function name, 
    "args": dictionary of argument name and its value,
}}

You SHOULD NOT include any other text in the response if you call a function
retrieve(input_query: str) -> str - Retrieve useful and important documents based on the input query.

This function is intended to interact with a retrieval system, such as a vector store,
keyword-based search, or document index, to return relevant content.




In [40]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
# chain.invoke({"input": "what's thirteen times 4 and 12 plus 7"})
chain.invoke({"input": "hi there how are you? can you tell me what is task decomposition "})

{'name': 'retrieve', 'args': {'input_query': 'what is task decomposition?'}}

In [16]:
import uuid

class JsonOrRawParser(JsonOutputParser):
    def invoke(self, input, config=None):
        try:
            input.tool_calls = super().invoke(input) | {"id": str(uuid.uuid4()), "type": "tool_call"}
            input.content = ""
            return input
            
        except Exception as e:
            return input

In [17]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOrRawParser()
result = chain.invoke({"input": "2+3"})
result

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gemma3:27b', 'created_at': '2025-05-21T14:55:10.428253309Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1151981790, 'load_duration': 98748169, 'prompt_eval_count': 124, 'prompt_eval_duration': 40847117, 'eval_count': 41, 'eval_duration': 1010048795, 'model_name': 'gemma3:27b'}, id='run--42303f37-b6c8-4707-b6d6-f0375952234e-0', tool_calls={'name': 'add', 'args': {'x': 2, 'y': 3}, 'id': '417cfc70-be50-4637-8f7f-50deba18642d', 'type': 'tool_call'}, usage_metadata={'input_tokens': 124, 'output_tokens': 41, 'total_tokens': 165})

In [18]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOrRawParser()
chain.invoke({"input": "hi"})

AIMessage(content='hi there! how can i help you today?\n\n\n\n', additional_kwargs={}, response_metadata={'model': 'gemma3:27b', 'created_at': '2025-05-21T14:55:13.643496814Z', 'done': True, 'done_reason': 'stop', 'total_duration': 430958687, 'load_duration': 98520932, 'prompt_eval_count': 122, 'prompt_eval_duration': 35588876, 'eval_count': 12, 'eval_duration': 294855729, 'model_name': 'gemma3:27b'}, id='run--c381627a-fa7a-4a4d-9d69-c3d36fed6a1a-0', usage_metadata={'input_tokens': 122, 'output_tokens': 12, 'total_tokens': 134})

In [19]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply', 'args': {'a': 13, 'b': 4.14137281}, 'output': 53.83784653}

# multi-tool calling

In [6]:
# [
#   {{{{
#     "name": "tool_name",
#     "arguments": {{{{
#       "x": value1,
#       "y": value2
#     }}}}
#   }}}},
#   ...
# ]

# [{{{{"name": function name, "arguments": }}}}, {{{{...}}}}]

In [7]:
system_prompt = f"""\
You have access to functions. If you decide to invoke any of the function(s),
you MUST put it in the format of

[
  {{{{
    "name": "tool_name",
    "arguments": dictionary of argument name and its value
  }}}},
  ...
]

You SHOULD NOT include any other text in the response if you call a function.
{rendered_tools}
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [9]:
import uuid

class JsonOrRawParser(JsonOutputParser):
    def invoke(self, input, config=None):
        try:
            tool_calls = super().invoke(input)
            tool_calls = [tool_call | {"id": str(uuid.uuid4()), "type": "tool_call"} for tool_call in tool_calls]
            input.tool_calls = tool_calls
            input.content = ""
            
            return input
            
        except Exception as e:
            print(e)
            return input

In [10]:
chain = prompt | model | JsonOrRawParser()
chain.invoke({"input": "what's thirteen times 4 "})

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gemma3:27b', 'created_at': '2025-05-21T14:50:15.644604347Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7642959938, 'load_duration': 5954346745, 'prompt_eval_count': 139, 'prompt_eval_duration': 342502896, 'eval_count': 52, 'eval_duration': 1282938783, 'model_name': 'gemma3:27b'}, id='run--88737e0f-018a-486e-862e-49474440e357-0', tool_calls=[{'name': 'multiply', 'arguments': {'a': 13.0, 'b': 4.0}, 'id': 'dbfc737e-8135-43a5-bf51-3c5a52535454', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 52, 'total_tokens': 191})

In [11]:
from typing import Union, Any

def run_tool_calls(tool_calls: list[ToolCallRequest]):
    results = []
    for call in tool_calls:
        result = invoke_tool(call)
        results.append({
            "name": call["name"],
            "arguments": call["arguments"],
            "output": result
        })
    return results


def maybe_invoke_tool(response: Union[str, Any]):
    try:
        text = response.content if hasattr(response, "content") else response
        parsed = json.loads(text)
        print(parsed)
        if isinstance(parsed, dict) and "name" in parsed and "arguments" in parsed:
            # Single tool call
            output = invoke_tool(parsed)
            return {
                "name": parsed["name"],
                "arguments": parsed["arguments"],
                "output": output
            }
        else:
            # Not a tool call - just return text
            return {"output": text}
    except Exception:
        # Parsing failed, return raw response
        text = response.content if hasattr(response, "content") else response
        return {"output": text}

In [12]:
from langchain_core.runnables import RunnableLambda

chain = prompt | model | RunnableLambda(maybe_invoke_tool) | RunnableLambda(run_tool_calls)

In [13]:
chain.invoke({"input": "what is 234 * 3 and what is 293 + 98"})

TypeError: string indices must be integers, not 'str'

# add summarizer

In [34]:
from langchain_core.prompts import ChatPromptTemplate

summarize_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Summarize the tool results in simple, natural language. "
        "Avoid verbose introductions. Just clearly state what was computed."
    ),
    (
        "user",
        "Tool results:\n{results}"
    )
])

summarize_chain = summarize_prompt | model

In [35]:
from langchain_core.runnables import RunnableLambda

# The full chain: parse → invoke → summarize
chain = (
    prompt
    | model
    | JsonOutputParser()
    | RunnableLambda(run_tool_calls)
    | RunnableLambda(lambda state: summarize_chain.invoke({"results": state}))
    | RunnableLambda(lambda x: x.content if hasattr(x, "content") else x)
)


In [39]:
chain.invoke({"input": "what is 234 * 3 and 293 + 98"})

'234 multiplied by 3 equals 702. \n\n293 plus 98 equals 391.\n\n\n\n'